In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import plotly.graph_objects as go               # More plotly libs

## Testing using Hotel Dataset

In [2]:

# with open("illinois/factorization_data/measurement_matrix.txt") as file:
#     content = file.readlines()

In [3]:
# for i in range(len(content)):
#     row = content[i]
#     row = row.strip().split()
#     row = list(map(lambda x: float(x), row))
#     content[i] = row

In [4]:
def read_measurement_matrix(path):
    with open(path) as file:
        content = file.readlines()  

    for i in range(len(content)):
        row = content[i]
        row = row.strip().split()
        row = list(map(lambda x: float(x), row))
        content[i] = row
    return np.array(content)

In [5]:
W_alternating = read_measurement_matrix("media/factorization_data/measurement_matrix.txt")
num_frames, num_features = W_alternating.shape
num_frames//=2
num_frames
# num_features

101

In [6]:
def group_ij(M):
    Wx = M[::2]
    Wy = M[1::2]
    num_frames = M.shape[0]//2
    W =np.zeros_like(M)
    W[:num_frames] = Wx
    W[num_frames:] = Wy
    return W

def alternate_ij(M):
    M_alternating = np.zeros_like(M)
    size = M.shape[0]
    assert size%2==0
    halfsize = size//2

    M_i = M[:halfsize]
    M_j = M[halfsize:]
    for i in range(halfsize):
        M_alternating[2*i] = M_i[i]
        M_alternating[2*i+1] = M_j[i]

    return M_alternating

In [7]:
W = group_ij(W_alternating)


In [8]:
def find_initial_guess(W):
    W_CENTERED = W - np.atleast_2d(W.mean(axis=1)).T
    U, SIG_FLAT, VT = np.linalg.svd(W_CENTERED, full_matrices=False)

    # change rank to 3
    U = U[:,:3]
    SIG = np.diag(SIG_FLAT)[:3,:3]
    VT = VT[:3,:]

    # Compute the initial guess for the motion and structure matrices
    MHAT = U @ np.sqrt(SIG)
    SHAT = np.sqrt(SIG) @ VT

    return MHAT, SHAT

In [9]:
MHAT, SHAT = find_initial_guess(W)

In [10]:
num_frames

101

In [11]:
MHAT_i = MHAT[:num_frames]
MHAT_j = MHAT[num_frames:]

In [12]:
def compute_QQT(MHAT:np.ndarray):
    num_frames = MHAT.shape[0]//2
    MHAT_i = MHAT[:num_frames]
    MHAT_j = MHAT[num_frames:]
    
    # We will find the entries of QQ^T by setting up a system of 
    # equations for each entry 
    A = np.zeros((3*num_frames, 6))
    b = np.zeros(3*num_frames)
    b[:2*num_frames] = 1



    for i in range(num_frames):

        # The proper i vectors have norm 1
        A[i, 0] = MHAT_i[i,0]**2
        A[i, 1] = 2 * MHAT_i[i,0]* MHAT_i[i,1]
        A[i, 2] = 2 * MHAT_i[i,0]* MHAT_i[i,2]
        A[i, 3] = MHAT_i[i,1]**2
        A[i, 4] = 2 * MHAT_i[i,2]* MHAT_i[i,2]
        A[i, 5] = MHAT_i[i,2]**2

        # The proper j vectors have norm 1

        A[num_frames + i, 0] = MHAT_j[i,0]**2
        A[num_frames + i, 1] = 2 * MHAT_j[i,0]* MHAT_j[i,1]
        A[num_frames + i, 2] = 2 * MHAT_j[i,0]* MHAT_j[i,2]
        A[num_frames + i, 3] = MHAT_j[i,1]**2
        A[num_frames + i, 4] = 2 * MHAT_j[i,2]* MHAT_j[i,2]
        A[num_frames + i, 5] = MHAT_j[i,2]**2

        # The i and j vectors are orthogonal and thus the dot is 0
        A[2 * num_frames + i, 0] = MHAT_i[i,0] * MHAT_j[i,0]
        A[2 * num_frames + i, 1] = MHAT_i[i,0] *  MHAT_j[i,1] + MHAT_i[i,1] *  MHAT_j[i,0]
        A[2 * num_frames + i, 2] = MHAT_i[i,0] *  MHAT_j[i,2] + MHAT_i[i,2] *  MHAT_j[i,2]
        A[2 * num_frames + i, 3] = MHAT_i[i,1] * MHAT_j[i,1]
        A[2 * num_frames + i, 4] = MHAT_i[i,1] *  MHAT_j[i,2] + MHAT_i[i,2] *  MHAT_j[i,1]
        A[2 * num_frames + i, 5] = MHAT_i[i,2] * MHAT_j[i,2]
    
    qqt_entries, residuals, rank, singular_vals = np.linalg.lstsq(A,b, rcond= None)


    # Assemble QQT
    QQT = np.zeros((3,3))

    QQT[0,0] = qqt_entries[0]
    QQT[1,1] = qqt_entries[3]
    QQT[2,2] = qqt_entries[5]

    QQT[0,1] = qqt_entries[1]
    QQT[1,0] = qqt_entries[1]

    QQT[0,2] = qqt_entries[2]
    QQT[2,0] = qqt_entries[2]

    QQT[1,2] = qqt_entries[4]
    QQT[2,1] = qqt_entries[4]

    return QQT, residuals, rank, singular_vals
    



In [13]:
QQT, residuals, rank, singular_vals = compute_QQT(MHAT)

In [14]:
residuals

array([0.18373474])

In [15]:
QQT

array([[6.35596288e-03, 6.49826467e-05, 1.06886461e-04],
       [6.49826467e-05, 7.18744988e-03, 6.64723700e-05],
       [1.06886461e-04, 6.64723700e-05, 1.56337101e-03]])

In [16]:
# Test whether the QQT actually works
print(MHAT_i[5].T @ QQT @ MHAT_i[5])
print(MHAT_j[2].T @ QQT @ MHAT_j[2])
print(MHAT_i[2].T @ QQT @ MHAT_j[2])
# Good enough, 

0.9365265706564595
0.9679660695320583
0.0004079116583850517


In [17]:
# Find Q and compute the actual object points

Q = np.linalg.cholesky(QQT)
QINV = np.linalg.inv(Q)

# Assemble final guess for Motion and Structure.
M = MHAT @ Q
S = QINV @ SHAT



In [18]:
# test whether the norms of the motion matrix are close to 1
np.linalg.norm(M, axis=1).mean()

0.9989407455332266

In [19]:
def compute_object_points(SHAT, QQT=None, Q=None, normalize_size = False):
    if QQT is None and Q is None:
        print("You must give either the QQT or the Q")

    if Q is None:
        Q = np.linalg.cholesky(QQT)
    
    QINV = np.linalg.inv(Q)

    # Assemble final guess for Motion and Structure.
    S = QINV @ SHAT
    
    if normalize_size:
        S = S / np.linalg.norm(S, axis=1).mean()
    return S

In [20]:
S = compute_object_points(SHAT, QQT)

In [21]:
def normalize_object_size(S):
    return S / np.linalg.norm(S, axis=1).mean()

In [22]:
def visualize_points(S):
    X = S[0]
    Y = S[1]
    Z = S[2]

    object_points = go.Scatter3d(x=X, y=Y, z=-Z, mode='markers', marker=dict(size=2, color=-Z, colorscale='Viridis', opacity=0.8))

    fig = go.Figure(data=[object_points])
    fig.update_layout(title='3D Point Cloud from TK Factorization', autosize=False, width=800, height=800, margin=dict(l=65, r=50, b=65, t=90))
    fig.show()


In [23]:
visualize_points(S)

In [24]:
X = S[0]
Y = S[1]
Z = S[2]

## One function to rule them all!

## Testing on Synthetic data

In [42]:
from plyfile import PlyData

In [64]:
# read PLY file 
def read_from_ply(path):
    ply_data = PlyData.read(path)
    vertices = ply_data["vertex"]
    X = vertices["x"]
    Y = vertices["y"]
    Z = vertices["z"]
    return np.vstack((X,Y,Z)).T

In [65]:
bunny_points = read_from_ply("bunny.ply")
bunny_points /= np.linalg.norm(bunny_points, axis=1).mean()

In [66]:
# A = np.random.uniform(-1, 1, (3, 3))  # Generate random matrix A
# Q = np.dot(A, A.T)  # Q is a symmetric matrix
# # Q is decomposed into RDR^T, where R is a rotation matrix
# R = np.linalg.svd(Q)[0]

In [67]:
# R

In [68]:
# np.linalg.norm(R[1])

In [107]:

def get_random_camera():

    I = np.random.uniform(-1, 1, (3,))
    J = np.random.uniform(-1, 1, (3,))

    I/= np.linalg.norm(I)
    
    K = np.cross(I,J)
    J = np.cross(K, I)
    J/= np.linalg.norm(J)

    # get random translation
    t = np.random.uniform(-1, 1, size=(3,))
    t = t / np.linalg.norm(t)
    
    return I,J
    # return np.vstack((I,J,K)), t


In [108]:
i,j = get_random_camera()

In [109]:
np.linalg.norm(j)

1.0

In [110]:
i

array([ 0.55930523, -0.10332167,  0.82249759])

In [111]:
bunny_points[0]

array([-0.28825286,  1.1660994 ,  0.04754027], dtype=float32)

In [112]:
np.dot(bunny_points, i)

array([-0.24260292, -0.31254116, -0.17670317, -0.01945973, -0.1713486 ,
        0.31121931,  0.23832947,  0.04628815,  0.18678388, -0.61135795,
        0.18087737, -0.3743288 , -0.20901929, -0.07895142,  0.04112229,
        0.15731867,  0.06739445, -0.4703165 , -0.28947043, -0.36023423,
       -0.32941712,  0.17686636, -0.2645987 , -0.13917124,  0.05599926,
       -0.16599484, -0.55639511,  0.10899457, -0.53231816, -0.38244931,
       -0.22490171,  0.1174409 , -0.41711499, -0.45395484,  0.11977945,
       -0.21570495,  0.22920576, -0.36898592, -0.53647125, -0.1951099 ,
        0.27337987,  0.3418775 , -0.31048826, -0.31333988, -0.30151617,
       -0.45197362, -0.26742375,  0.24584046,  0.08669016, -0.66805802,
        0.11515234,  0.38606061, -0.35556711, -0.87001452, -0.51585347,
        0.24329231,  0.28161474,  0.3469332 , -0.38927624, -0.35736449,
       -0.12627413, -0.07657476, -0.25640002, -0.42876846, -0.01152911,
       -0.67261995, -0.32644283,  0.06347824,  0.1667509 ,  0.00

In [115]:
def project_to_camera(points, i,j):
    
    image_points_i = np.dot(points, i)
    image_points_j = np.dot(points, j)
    image_points = np.vstack((image_points_i, image_points_j))
    return image_points.T

In [116]:
project_to_camera(bunny_points, *get_random_camera()).shape

(453, 2)

In [118]:
W = []
num_frames = 40
for i in range(num_frames):
    camera = get_random_camera()
    points = project_to_camera(bunny_points, *camera).T
    points -= np.mean(points, axis=0, keepdims=True)
    W.append(points)
W = np.vstack(W)

In [119]:
W = group_ij(W)

In [120]:
MHAT, SHAT = find_initial_guess(W)

In [121]:
QQT, residuals, rank, singular_vals = compute_QQT(MHAT)

In [122]:
residuals

array([19.10684996])

In [123]:
MHAT_i = MHAT[:num_frames]
MHAT_j = MHAT[num_frames:]

In [124]:
# Test whether the QQT actually works
print(MHAT_i[5].T @ QQT @ MHAT_i[5])
print(MHAT_j[2].T @ QQT @ MHAT_j[2])
print(MHAT_i[2].T @ QQT @ MHAT_j[2])

0.9356046703602967
0.04464662297877142
-0.044646622978771325


### Viewpoints

In [36]:
# Get camera locations as one away from the origin
def get_camera_locations(M, alternating=False, object_size = 1):
    x = np.array([object_size, 0, 0])
    
    if not alternating:
        M = alternate_ij(M)

    
    num_frames = M.shape[0] // 2
    camera_locations = np.zeros((num_frames,3))

    for i, camera in enumerate(np.split(M, num_frames)):
        normal = np.cross(camera[0],camera[1])
        R = np.vstack((camera, normal))

        camera_locations[i] = np.dot(R,x)

        
    return camera_locations
